- use "base" env, not "py37"!!

In [1]:
# !pip install psutil

In [2]:
# import psutil

# # Get the memory details
# memory = psutil.virtual_memory()

# # Available RAM in bytes
# available_ram_bytes = memory.available

# # Convert bytes to GB for easier interpretation
# available_ram_gb = available_ram_bytes / (1024 ** 3)

# print(f"Available RAM: {available_ram_gb:.2f} GB")


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

## sentiment

In [4]:
from transformers import pipeline

In [5]:
# model_path = "./my_local_model_directory_oldv/" 
model_path = "./my_local_model_directory/" #"./my_local_model_directory_oldv/" #"cardiffnlp/twitter-roberta-base-sentiment" # for comparison

In [6]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

In [9]:
param_name = 'chargeSpeed'
expt_type = 'playfight/without_cover_story/'
rootfile_loc = f'../../data/text_responses/{expt_type}'

In [10]:
df =  pd.read_csv(f'{rootfile_loc}/data_triallevel_sorted.csv',index_col=[0])
#excluded_words_list = ['dot','around','dont know','black','grey', 'gray', 'circle', 'color', 'dot', 'shape', 'one', 'ball', "don't know", 'kids']

In [11]:
words_to_compare = df['responses'].to_list()
words_to_compare = [s.lower() for s in words_to_compare]
words_to_compare

['collided and then bounced off each other',
 'the dots moved apart, and then came back together, in a slow, fluid motion over and over',
 'softy hit twice',
 'the dots wandered and came together almost incidentally the first time, and then intentionally the second time.',
 'softer collisons',
 'slow coming together',
 'slowly connect',
 "dot moves slowly around. hit each other twice. didn't touch edges",
 'the dots slowly gravitate towards eachother and collide then bounce away again',
 'meandering',
 'slow collision ',
 'crash',
 'collision',
 'the dots seemed to move more gently this time and almost met each other gently twice instead of colliding with each other this time.',
 'the two dots hit eachother twice and they did it very slowly',
 'the dots alternated at a normal speed between pulling apart and drawing together',
 'touches',
 'slow speed bumper cars',
 'the dots moved a little slower & moved together twice.',
 'the grey dot slowly swept towards the black dot, the black red

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [12]:
# Encode the reference phrase and the words to compare
sentiment_scores =  np.full((df.shape[0],3),np.nan)

# Compare and print similarities
for i,word in enumerate(words_to_compare):
    encoded_input = tokenizer(word, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    sentiment_scores[i,:] = scores 
df['neg_score']  = sentiment_scores[:,0]
df['neu_score']  = sentiment_scores[:,1]
df['pos_score']  = sentiment_scores[:,2]
df.head()

,subID,stimset_rows,chargeSpeed,stim_dur,responses,movie,trial_num,neg_score,neu_score,pos_score
1,28000,26,1.5,8234,collided and then bounced off each other,29e7c94f-a6bf-4f1d-8ea1-46efbdb6a836,1,0.206024,0.762335,0.031641
1,28001,25,1.5,8130,"The dots moved apart, and then came back toget...",e838dadb-c02f-4273-ae93-210720dcd06c,1,0.034780,0.890654,0.074566
3,28002,28,1.5,8325,softy hit twice,b3a507ff-2289-4ad3-8922-1a3d0532e154,3,0.054410,0.689640,0.255951
6,28003,29,1.5,8085,The dots wandered and came together almost inc...,1b1159fc-de2b-4ae5-a736-d98d8524a366,6,0.049825,0.873125,0.077049
4,28004,32,1.5,8069,softer collisons,1f6c6525-a740-48c2-8dfc-9c4fb520c630,4,0.097854,0.798465,0.103681


In [13]:
df.sort_values(by = ['chargeSpeed','neg_score','neu_score','pos_score'],inplace=True)
df.head()

,subID,stimset_rows,chargeSpeed,stim_dur,responses,movie,trial_num,neg_score,neu_score,pos_score
5,28024,65,1.5,7962,now they slowly met and touched twice,1b1159fc-de2b-4ae5-a736-d98d8524a366,5,0.003994,0.731555,0.264451
3,28022,59,1.5,8002,The dots meet each other warmly.,1a8cf3c1-690b-449c-bb36-fd3b251837c1,3,0.004204,0.169153,0.826644
3,28083,46,1.5,8083,both meeting each other,50919f52-6a9c-4824-b24f-2f1aebd58e1b,3,0.005532,0.861102,0.133366
1,28050,98,1.5,8648,slowly came together,8ed7c0fc-1490-4e35-a000-7f7357a0772a,1,0.006125,0.355109,0.638766
1,28070,26,1.5,8284,The dots moved to the left and down and collid...,29e7c94f-a6bf-4f1d-8ea1-46efbdb6a836,1,0.006636,0.873458,0.119906


In [14]:
df['pos-neg'] = df['pos_score'] - df['neg_score']
df['pos-neu'] = df['pos_score'] - df['neu_score']
df['neg-neu'] = df['neg_score'] - df['neu_score']
df['neu-neg'] = df['neu_score'] - df['neg_score']
df.to_csv(f'{rootfile_loc}/sentiment_analyses/data_with_sentiment_scores.csv')

In [15]:
len(np.unique(df['subID']))

103